In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!nvcc --version

Fri May 24 16:50:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.02              Driver Version: 536.19       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070        On  | 00000000:02:00.0 Off |                  N/A |
|  0%   43C    P8               9W / 220W |      0MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath('../'))
import mitsuba as mi
import torch
from matplotlib import pyplot as plt
import drjit as dr
from utils_mitsuba import setup_shadowscene
device = 'cpu'
if torch.cuda.is_available():
    print("using cuda")
    device = 'cuda'
    mi.set_variant('cuda_ad_rgb')
# device = 'cpu'
# mi.set_variant('scalar_rgb')
mi.variants()

using cuda


['scalar_rgb', 'scalar_spectral', 'cuda_ad_rgb', 'llvm_ad_rgb']

In [8]:
img=mi.render(mi.load_dict(mi.cornell_box()))
plt.axis("off")
plt.imshow(img ** (1.0 / 2.2)); # approximate sRGB tonemapping
mi.util.write_bitmap("cbox_test.png", img)

RuntimeError: jit_init_thread_state(): the CUDA backend hasn't been initialized. Make sure to call jit_init(JitBackend::CUDA) to properly initialize this backend.

In [16]:
hparams = {'resx': 256,
            'resy': 192,
            'nsamples': 1,
            'sigma': 0.5,
            'render_spp': 32,
            'initial_translation': [-0.5, 2.5],
            'gt_translation': [-1.5, 1.0],
            'learning_rate': 2e-2,
            'epochs': 400,
            'sigma_annealing': True,
            'anneal_const_first': 200,
            'anneal_const_last': 0,
            'anneal_sigma_min': 0.01,
            'integrator': 'path',
            'max_depth': 6,
            'reparam_max_depth': 2}
scene, params, mat_id, initial_vertex_positions = setup_shadowscene(hparams)

/mnt/d/ucl cs/MSc/prdpt2024


Exception: unravel(): expected a nested array as target type!

In [7]:
# scene = mi.load_file('./scenes/shadows/shadows.xml')
img=mi.render(scene)
plt.axis("off")
plt.imshow(img ** (1.0 / 2.2)); # approximate sRGB tonemapping
mi.util.write_bitmap("shadows.png", img)

RuntimeError: ​[xml.cpp:179] Error while loading "./scenes/shadows/shadows.xml" (at line 12, col 4): could not parse integer value "depth_value".